In [2]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import gc, sys, os

In [3]:
df_member = pd.read_csv('../Member.csv')
df_ecoupon = pd.read_csv('../ECoupon.csv')
df_promotion = pd.read_csv('../PromotionOrders.csv')
df_main = pd.read_csv('../Orders.csv')
df_main.info()

D:\andyl\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
D:\andyl\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5495276 entries, 0 to 5495275
Data columns (total 28 columns):
DateId                         int64
MemberId                       int64
OrderGroupCode                 object
TrackSourceTypeDef             object
TrackDeviceTypeDef             object
PayProfileTypeDef              object
SalesOrderSlaveId              int64
SalePageId                     int64
IsMajor                        bool
IsGift                         bool
IsSalePageGift                 bool
Quantity                       int64
UnitPrice                      float64
PromotionDiscount              float64
ECouponId                      int64
ECouponDiscount                float64
SalesOrderSlaveTotalPayment    float64
SalesOrderSlaveDateTime        object
SalesOrderReceiverId           int64
City                           object
District                       object
ZipCode                        float64
StoreName                      object
StoreId              

In [4]:
df_main['StatusDef'].replace(['CreditCheckFail','WaitingToShipping'],np.NaN,inplace=True)

In [5]:
df_main['PayProfileTypeDef'].replace('ATM',np.NaN,inplace=True)

DISCARD_CITY = ['<fo', 'Kao', 'Kot', 'Tao','Hy ', 'New', '25 ', 'Keb', '1st', 'Tai', 'Ins']
KAO_AREA = ['高雄市', '屏東縣', '屏東市']
BIG_TAIPEI= ['新北市', '宜蘭縣', '基隆市']
THM = ['桃園市','新竹市','苗栗縣', '新竹縣']
MID = ['台中市','彰化縣', '南投縣']
MID_SOUTH = ['嘉義縣','台南市' ,'雲林縣','嘉義市']
Others = ['花蓮縣','台東縣', '台東市', '澎湖縣']
df_main['City'].replace(DISCARD_CITY,np.NaN,inplace=True)
df_main['City'].replace(KAO_AREA,'高坪',inplace=True)
df_main['City'].replace(BIG_TAIPEI,'新北周遭',inplace=True)
df_main['City'].replace(THM,'桃竹苗',inplace=True)
df_main['City'].replace(MID,'中彰投',inplace=True)
df_main['City'].replace(MID_SOUTH,'雲嘉南',inplace=True)
df_main['City'].replace(Others,'其他',inplace=True)

df_main['StatusDef'].replace(['CreditCheckFail','WaitingToShipping'],np.NaN,inplace=True)
df_main['CauseDef'].replace(['其他', '商品有瑕疵或故障','價格比較貴', '商品寄錯或缺件', '送貨速度太慢'],np.NaN,inplace=True)

df_main['City'].unique()

array(['雲嘉南', '桃竹苗', '新北周遭', '台北市', '中彰投', '高坪', '其他', nan, '桃園縣', 'Đại',
       'Cha', 'Xin', 'Quậ', 'Nan', 'No.', 'Hsi', 'Daf', 'Aya', 'Jia'],
      dtype=object)

In [6]:
df_main['StatusDef'].value_counts()

Finish         4185534
Cancel          749122
Overdue         293402
ReturnGoods     264761
Name: StatusDef, dtype: int64

In [7]:
EC_DROP=['ECouponStartDateTime',
       'ECouponEndDateTime', 'ECouponUsingStartDateTime',
       'ECouponUsingEndDateTime', 'ECouponDiscountPrice',
       'ECouponMaxDiscountLimit']
df_ecoupon.drop(EC_DROP,axis=1,inplace=True,errors='ignore')
df_ecoupon.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 2 columns):
ECouponId         261 non-null int64
ECouponTypeDef    261 non-null object
dtypes: int64(1), object(1)
memory usage: 4.2+ KB


In [8]:
PR_DROP=['PromotionId', 'SalesOrderSlavePromotionDiscount',
       'SalesOrderSlavePromotionDiscountAllocated',
       'SalesOrderSlavePromotionDiscountAllocatedQty',
       'SalesOrderSlavePromotionDiscountRate',
       'PromotionCondition_DiscountPrice', 'PromotionCondition_DiscountRate',
       'PromotionCondition_TotalPrice', 'PromotionCondition_TotalQty',
       'PromotionStartDateTime', 'PromotionEndDateTime']
df_promotion.drop(PR_DROP,axis=1,inplace=True,errors='ignore')
df_promotion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4011064 entries, 0 to 4011063
Data columns (total 2 columns):
SalesOrderSlaveId    int64
DiscountType         object
dtypes: int64(1), object(1)
memory usage: 61.2+ MB


In [9]:
df_main = df_main.merge(df_ecoupon, on='ECouponId', how='left')
df_main = df_main.merge(df_promotion, on='SalesOrderSlaveId', how='left')

In [10]:
DISCARD_LABEL = ['Unnamed: 0',
       'SalesOrderSlaveId', 'SalePageId', 'IsGift',
       'IsSalePageGift',
       'ECouponId',
       'SalesOrderSlaveDateTime', 'SalesOrderReceiverId', 'District',
       'ZipCode', 'StoreName', 'StoreId', 'ReturnGoodsDateId']
df_main.drop(DISCARD_LABEL,axis=1,inplace=True,errors='ignore')
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6678387 entries, 0 to 6678386
Data columns (total 18 columns):
DateId                         int64
MemberId                       int64
OrderGroupCode                 object
TrackSourceTypeDef             object
TrackDeviceTypeDef             object
PayProfileTypeDef              object
IsMajor                        bool
Quantity                       int64
UnitPrice                      float64
PromotionDiscount              float64
ECouponDiscount                float64
SalesOrderSlaveTotalPayment    float64
City                           object
DeliverTypeDef                 object
StatusDef                      object
CauseDef                       object
ECouponTypeDef                 object
DiscountType                   object
dtypes: bool(1), float64(4), int64(3), object(10)
memory usage: 923.5+ MB


In [11]:
main_dummies=pd.get_dummies(df_main,columns=['TrackSourceTypeDef','TrackDeviceTypeDef', 'PayProfileTypeDef',
                                             'City', 'DeliverTypeDef', 'StatusDef','CauseDef', 'ECouponTypeDef', 'DiscountType'])

In [12]:
main_dummies['Date'] = pd.to_datetime(main_dummies['DateId'].astype(str), format='%Y%m%d')
main_dummies.drop(labels=['DateId','new_DateId'], inplace=True, axis=1,errors='ignore')
main_dummies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6678387 entries, 0 to 6678386
Data columns (total 62 columns):
MemberId                            int64
OrderGroupCode                      object
IsMajor                             bool
Quantity                            int64
UnitPrice                           float64
PromotionDiscount                   float64
ECouponDiscount                     float64
SalesOrderSlaveTotalPayment         float64
TrackSourceTypeDef_AndroidApp       uint8
TrackSourceTypeDef_Web              uint8
TrackSourceTypeDef_iOSApp           uint8
TrackDeviceTypeDef_Mobile           uint8
TrackDeviceTypeDef_PC               uint8
TrackDeviceTypeDef_Pad              uint8
PayProfileTypeDef_CreditCardOnce    uint8
PayProfileTypeDef_Family            uint8
PayProfileTypeDef_LinePay           uint8
PayProfileTypeDef_SevenEleven       uint8
City_Aya                            uint8
City_Cha                            uint8
City_Daf                            uin

In [13]:
t_begin =main_dummies['Date'].min()
t_range = pd.Timedelta(183, unit='d')
print(t_begin)

2015-05-06 00:00:00


In [14]:
SEASON_CNT = 2
df_mains = {}
for i in range(SEASON_CNT):
    df_mains[i] = \
        main_dummies[main_dummies['Date'] < t_begin + t_range * (i + 1)]\
        [main_dummies['Date'] >= t_begin + t_range * (i)]

D:\andyl\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [15]:
df_mains[0].groupby('MemberId')['OrderGroupCode'].nunique()

MemberId
1329        3
1372        2
1373        1
1423        1
1469       11
1474        1
1498        1
1618        1
1938        1
2008        1
2054        1
2200        1
2723        1
2725        1
2940        1
2943        1
3028        1
3249        2
3270        1
3408        2
3722        1
3762        1
3921        1
3945        3
4045        1
4077        1
4120        2
4652        1
4797        1
4805        1
           ..
2129519     1
2129582     1
2129715     1
2129728     1
2129811     1
2129855     1
2129975     1
2130025     1
2130032     1
2130044     1
2130220     1
2130493     1
2130509     1
2130547     1
2130566     1
2130598     1
2130688     1
2130841     1
2130932     1
2130973     1
2130990     1
2131056     1
2131709     1
2131973     1
2132088     1
2132162     1
2718785     1
5977952     1
6374955     2
6486287     1
Name: OrderGroupCode, Length: 69883, dtype: int64

In [16]:
df_main_member = {}
for k in df_mains:
    df_main_member[k]=df_mains[k].groupby('MemberId').sum()
df_main_member[0]


,IsMajor,Quantity,UnitPrice,PromotionDiscount,ECouponDiscount,SalesOrderSlaveTotalPayment,TrackSourceTypeDef_AndroidApp,TrackSourceTypeDef_Web,TrackSourceTypeDef_iOSApp,TrackDeviceTypeDef_Mobile,...,ECouponTypeDef_DrawOut,ECouponTypeDef_FirstDownload,ECouponTypeDef_OpenCard,DiscountType_任選優惠價,DiscountType_會員滿額打折,DiscountType_滿件打折,DiscountType_滿件折現,DiscountType_滿額打折,DiscountType_滿額折現,DiscountType_滿額贈
MemberId,,,,,,,,,,,,,,,,,,,,,
1329,3.0,3,918.0,0.0,0.0,918.0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1372,2.0,2,250.0,0.0,0.0,250.0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1373,3.0,3,648.0,-150.0,-50.0,448.0,0,0,3,3,...,3,0,0,2,0,0,0,0,0,0
1423,2.0,3,273.0,0.0,0.0,373.0,2,0,0,2,...,0,0,0,0,0,0,0,0,0,0
1469,30.0,30,7193.0,-499.0,-259.0,6435.0,30,0,0,30,...,7,0,3,2,0,0,0,0,5,0
1474,3.0,3,1248.0,0.0,0.0,1248.0,0,3,0,3,...,0,0,0,0,0,0,0,0,0,0
1498,6.0,6,1321.0,-100.0,-50.0,1171.0,0,0,6,6,...,0,0,6,0,0,0,0,0,6,0
1618,1.0,1,399.0,0.0,0.0,399.0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1938,1.0,1,375.0,0.0,0.0,375.0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [17]:
main_by_baskets = {}
for k in df_mains:
    main_by_baskets[k] = df_mains[k]\
        .groupby(by=['OrderGroupCode','MemberId'], as_index=False)\
            ['SalesOrderSlaveTotalPayment']\
        .sum()
    print(main_by_baskets[k].info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98999 entries, 0 to 98998
Data columns (total 3 columns):
OrderGroupCode                 98999 non-null object
MemberId                       98999 non-null int64
SalesOrderSlaveTotalPayment    98999 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 219893 entries, 0 to 219892
Data columns (total 3 columns):
OrderGroupCode                 219893 non-null object
MemberId                       219893 non-null int64
SalesOrderSlaveTotalPayment    219893 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.7+ MB
None


In [18]:
member_avgs = {}
def rename_col_to(name, old, new):
    if name == old:
        return new
    else:
        return name

for k in main_by_baskets:
    member_avgs[k] = main_by_baskets[k]\
        .groupby(by='MemberId', as_index=False)\
        .agg('mean')
    member_avgs[k].rename(columns = lambda r: rename_col_to(r, 'SalesOrderSlaveTotalPayment', 'RecentAveragePurchase'), inplace=True)
    print(member_avgs[k].head())

   MemberId  RecentAveragePurchase
0      1329                  306.0
1      1372                  125.0
2      1373                  448.0
3      1423                  373.0
4      1469                  585.0
   MemberId  RecentAveragePurchase
0      1334             333.000000
1      1373             143.714286
2      1384             557.000000
3      1391             593.000000
4      1393             199.000000


In [19]:
member_purchase_cnts = {}
for k in main_by_baskets:
    member_purchase_cnts[k] = main_by_baskets[k]\
        .groupby(by='MemberId', as_index=False)\
        .agg('count')
    print(member_purchase_cnts[k].info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69883 entries, 0 to 69882
Data columns (total 3 columns):
MemberId                       69883 non-null int64
OrderGroupCode                 69883 non-null int64
SalesOrderSlaveTotalPayment    69883 non-null int64
dtypes: int64(3)
memory usage: 2.1 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 139516 entries, 0 to 139515
Data columns (total 3 columns):
MemberId                       139516 non-null int64
OrderGroupCode                 139516 non-null int64
SalesOrderSlaveTotalPayment    139516 non-null int64
dtypes: int64(3)
memory usage: 4.3 MB
None


In [20]:
member_purchase_medians = {}
member_purchase_averages = {}
for k in member_purchase_cnts:
    member_avgs[k]['RecentCount'] = member_purchase_cnts[k]['OrderGroupCode']
    member_avgs[k]['RecentTotal'] = member_avgs[k]['RecentCount'] * member_avgs[k]['RecentAveragePurchase']
    member_purchase_medians[k] = member_avgs[k]['RecentAveragePurchase'].median()
    member_purchase_averages[k] = member_avgs[k]['RecentAveragePurchase'].mean()
    print(member_avgs[k].head())
    print(member_purchase_medians[k])

   MemberId  RecentAveragePurchase  RecentCount  RecentTotal
0      1329                  306.0            3        918.0
1      1372                  125.0            2        250.0
2      1373                  448.0            1        448.0
3      1423                  373.0            1        373.0
4      1469                  585.0           11       6435.0
694.0
   MemberId  RecentAveragePurchase  RecentCount  RecentTotal
0      1334             333.000000            3        999.0
1      1373             143.714286            7       1006.0
2      1384             557.000000            1        557.0
3      1391             593.000000            1        593.0
4      1393             199.000000            1        199.0
720.0


In [21]:
df_members = {}
for k in member_avgs:
    df_members[k] = df_member.merge(member_avgs[k], how='left')
    #print(df_members[k].info())
    #print(df_members[k].head())

In [22]:
RecentCount_threshold = 2
def set_member_group(row, axis_th_x = RecentCount_threshold, axis_th_y = 0, axis_key_x = 'RecentCount', axis_key_y = 'RecentAveragePurchase'):
    value = 1
    if not np.isnan(row[axis_key_x]) and row[axis_key_x] < axis_th_x:
        if row[axis_key_y] < axis_th_y:
            value = 4
        else:
            value = 3
    elif not np.isnan(row[axis_key_x]):
        if row[axis_key_y] < axis_th_y:
            value = 5
        else:
            value = 2
    
    return value

for k in df_members:
    df_members[k]['MemberGroup'] = df_members[k]\
        .apply(lambda r: set_member_group(r, RecentCount_threshold, member_purchase_medians[k]), axis = 1)

In [23]:
df_members[1].columns

Index(['MemberId', 'Birthday', 'GenderTypeDef', 'RegisterSourceTypeDef',
       'RegisterDateTime', 'MinOrderDate', 'OpenCardPresent',
       'LastBirthdayPresentYear', 'IsInBlackList', 'RecentAveragePurchase',
       'RecentCount', 'RecentTotal', 'MemberGroup'],
      dtype='object')

In [24]:
df_members[1]['MinOrderDate'].unique()

array([nan, '2018-02-07', '2018-03-13', ..., '2015-05-06', '2015-05-07',
       '2015-05-13'], dtype=object)

In [25]:
DISCARD_LABEL_MEMBER = [
    'Birthday',
    'RegisterDateTime',
    'MinOrderDate',
    'OpenCardPresent',
    'LastBirthdayPresentYear',
    'IsInBlackList',
]

for k in df_members:
    df_members[k].drop(labels=DISCARD_LABEL_MEMBER, inplace=True, axis=1, errors='ignore')

In [26]:
for k in df_members:
    df_main_member[k].reset_index(inplace=True)
    df_members[k] = df_members[k].merge(
        df_main_member[k],
        left_on='MemberId',
        right_on='MemberId',
        how='left')
    
df_members[0].info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 859146 entries, 0 to 859145
Data columns (total 66 columns):
MemberId                            859146 non-null int64
GenderTypeDef                       237276 non-null object
RegisterSourceTypeDef               829143 non-null object
RecentAveragePurchase               69533 non-null float64
RecentCount                         69533 non-null float64
RecentTotal                         69533 non-null float64
MemberGroup                         859146 non-null int64
IsMajor                             69533 non-null float64
Quantity                            69533 non-null float64
UnitPrice                           69533 non-null float64
PromotionDiscount                   69533 non-null float64
ECouponDiscount                     69533 non-null float64
SalesOrderSlaveTotalPayment         69533 non-null float64
TrackSourceTypeDef_AndroidApp       69533 non-null float64
TrackSourceTypeDef_Web              69533 non-null float64
TrackS

In [27]:
for k in df_members:
    df_members[k] = pd.get_dummies(df_members[k],drop_first=True)
    df_members[k].fillna(0,inplace=True)

df_members[0].head()

,MemberId,RecentAveragePurchase,RecentCount,RecentTotal,MemberGroup,IsMajor,Quantity,UnitPrice,PromotionDiscount,ECouponDiscount,...,DiscountType_任選優惠價,DiscountType_會員滿額打折,DiscountType_滿件打折,DiscountType_滿件折現,DiscountType_滿額打折,DiscountType_滿額折現,DiscountType_滿額贈,GenderTypeDef_Male,RegisterSourceTypeDef_Web,RegisterSourceTypeDef_iOSApp
0,1587633,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
1,1587634,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
2,1587658,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
3,1587809,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
4,1587830,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0


In [28]:
def replace_col_label(df, old_label, new_label):
    col = list(df.columns)
    for i in range(len(col)):
        if col[i] == old_label:
            col[i] = new_label
            df.columns = col
            return

for i in range(len(df_members)):
    if i == len(df_members) - 1:
        break
    df = df_members[i + 1].loc[:, ['MemberId', 'MemberGroup']]
    replace_col_label(df, 'MemberGroup', 'MemberGroupNext')
    print(df.shape)
    df_members[i] = df_members[i].merge(df, on='MemberId', how='left')
    df_members[i]['MemberGroupNext'].fillna(1, inplace=True)
    gc.collect()

#df_members[0].dtypes

(859146, 2)


In [45]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from imblearn.over_sampling import RandomOverSampler

CURRENT_NULL_GROUP_ENABLED = False
# True: Predict all 5 class
#False: Only predict if customer will buy anything in next season 
FULL_PREDICT = True 

TRAIN_FEATURES = [
    ##'MemberId',    
    'RecentAveragePurchase',    
    'RecentCount',    
    'RecentTotal',
    'IsMajor',    
    'Quantity',    
    'UnitPrice',
    'PromotionDiscount',
    'ECouponDiscount',
    'SalesOrderSlaveTotalPayment',
    ##'TrackSourceTypeDef_AndroidApp',    'TrackSourceTypeDef_Web',    'TrackSourceTypeDef_iOSApp',    'TrackDeviceTypeDef_Mobile',    'TrackDeviceTypeDef_PC',    'TrackDeviceTypeDef_Pad',
    ##'PayProfileTypeDef_CreditCardOnce',    'PayProfileTypeDef_Family',    'PayProfileTypeDef_LinePay',    'PayProfileTypeDef_SevenEleven',
    ###'City_Aya',    'City_Cha',    'City_Daf',    'City_Hsi',    'City_Jia',    'City_Nan',    'City_No.',    'City_Quậ',    'City_Xin',    'City_Đại',    
    'City_中彰投',    'City_其他',    'City_台北市',    'City_新北周遭',    'City_桃園縣',    'City_桃竹苗',    'City_雲嘉南',    'City_高坪',
    'DeliverTypeDef_Family',    'DeliverTypeDef_Home',    'DeliverTypeDef_SevenEleven',    
    'StatusDef_Cancel',    'StatusDef_Finish', 'StatusDef_Overdue',    'StatusDef_ReturnGoods',    
    ##'CauseDef_商品使用不如預期',    'CauseDef_商品質感/品質不佳',    'CauseDef_尺寸不合',    'CauseDef_改買了別款或改付款方式',    'CauseDef_規格不符、商品與網頁標示不清',    'CauseDef_重複購買或不需要了',
    ##'ECouponTypeDef_Birthday',    'ECouponTypeDef_Code',    'ECouponTypeDef_DrawOut',    'ECouponTypeDef_FirstDownload',    'ECouponTypeDef_OpenCard',
    'DiscountType_任選優惠價',    'DiscountType_會員滿額打折',    'DiscountType_滿件打折',    'DiscountType_滿件折現',    'DiscountType_滿額打折',    'DiscountType_滿額折現',    'DiscountType_滿額贈',
    ##'GenderTypeDef_Male',
    'RegisterSourceTypeDef_Web',    'RegisterSourceTypeDef_iOSApp',
    'MemberGroup',
    'MemberGroupNext'
]

rfcs = {}
rfc_models = {}
results_matrixs = {}
results = {}

test = pd.DataFrame()
for k in df_members:
    if k >= len(df_members) - 1:
        break
    #debug
    # todo convert Timestamp to int64
    #enddebug
    try:
        cache_df = df_members[k].copy()
        #debug
        #drop na where MemberGroupNext has nan value
        if not CURRENT_NULL_GROUP_ENABLED:
            cache_df = cache_df[cache_df['MemberGroup'] != 1]
        if not FULL_PREDICT:
            cache_df['MemberGroupNext'] = (cache_df['MemberGroupNext'] > 1)
            
        cache_df = cache_df[np.isfinite(cache_df['MemberGroupNext'])] 
        #print(cache_df.dtypes)
        #enddebug
        print('season %i shape: %r'%(k, cache_df.shape))
        
        train, test = train_test_split(
            cache_df, 
            test_size = 0.25, 
            random_state = 0
        )
        
        class_w = {}
        if FULL_PREDICT:
            class_w={
                1:0.1,
                2:1,
                3:1,
                4:1,
                5:1
            }
        else:
            class_w={
                True: 1,
                False: 0.1
            }
        
        rfcs[k] = RandomForestClassifier(
            n_estimators=500,
            class_weight=class_w
        )
        
        '''
        AdaBoostClassifier(
            n_estimators=10000
        )
        '''
        rfc_models[k] = rfcs[k].fit(
            train.drop(
                labels=['MemberId', 'MemberGroupNext'], 
                axis = 1,
                errors='ignore'), 
            train['MemberGroupNext']
        )
        test['predicted'] = rfc_models[k].predict(
            test.drop(
                labels=['MemberId', 'MemberGroupNext'], 
                axis = 1, 
                errors='ignore')
        )
        results_matrixs[k] = confusion_matrix(test['MemberGroupNext'], test['predicted'])
        results[k] = test
        accuracy = test.loc[
            :,
            ['predicted', 'MemberGroupNext']]\
            .apply(lambda r: r['predicted'] == r['MemberGroupNext'], axis =1)\
            .sum()/test.shape[0]
            
        print('in group %i:\naccuracy: %f'%(k, accuracy))
        print(test.loc[:, ['predicted', 'MemberGroupNext']])
        
        print('\nAns \ Pred')
        print(results_matrixs[k])
        
    except Exception as e:
        sys.stdout.write('Error occurs at season %i\n'%k)
        raise e
    finally:
        gc.collect()

season 0 shape: (69533, 68)


D:\andyl\Anaconda3\lib\site-packages\ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


in group 0:
accuracy: 0.654280
        predicted  MemberGroupNext
649640          1                1
858359          1                2
391232          1                1
811629          1                4
613797          1                1
149418          1                1
694090          1                1
653803          1                1
852922          1                1
553862          1                1
153511          1                1
556245          1                1
149364          1                5
682703          1                1
794625          1                3
648183          1                2
643541          1                1
148586          1                1
802912          1                5
276924          1                1
424574          1                1
562644          1                2
530684          1                1
691812          1                1
784099          1                1
605947          1                1
608267          1       

In [46]:
df_main['']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6678387 entries, 0 to 6678386
Data columns (total 18 columns):
DateId                         int64
MemberId                       int64
OrderGroupCode                 object
TrackSourceTypeDef             object
TrackDeviceTypeDef             object
PayProfileTypeDef              object
IsMajor                        bool
Quantity                       int64
UnitPrice                      float64
PromotionDiscount              float64
ECouponDiscount                float64
SalesOrderSlaveTotalPayment    float64
City                           object
DeliverTypeDef                 object
StatusDef                      object
CauseDef                       object
ECouponTypeDef                 object
DiscountType                   object
dtypes: bool(1), float64(4), int64(3), object(10)
memory usage: 1.2+ GB
